<a href="https://colab.research.google.com/github/inspire-lab/SecurePrivateAI/blob/master/1_intro/intro_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taken from the www.keras.io quickstart guide and their github page: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py and https://github.com/podschwadt/teaching

# Getting started: 30 seconds to Keras
The core data structure of Keras is a model, a way to organize layers. The simplest type of model is the Sequential model, a linear stack of layers. For more complex architectures, you should use the Keras functional API, which allows to build arbitrary graphs of layers.

Here is the `Sequential` model:



In [0]:
from keras.models import Sequential
import keras

model = Sequential()

Stacking layers is as easy as `.add()`:

In [0]:
from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=784))
model.add(Dense(units=10, activation='softmax'))

Once your model looks good, configure its learning process with `.compile()`:

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

If you need to, you can further configure your optimizer. A core principle of Keras is to make things reasonably simple, while allowing the user to be fully in control when they need to (the ultimate control being the easy extensibility of the source code).

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

Now that we have model we need some data to feed it. Let's load the MNIST hadnwritten digites and bring them into the correct form for our model ot use.

In [0]:
from keras.datasets import mnist
# the data, split between train and test sets

(x_train, y_train), (x_test, y_test) = mnist.load_data()

The image is loaded as integers. We need it to be floats between 0 and 1 though. On top of that our model only takes one dimensional arrays as input.

Squash the values to be in the interval `[0,1]` and reshape the individual to be one dimensional arrays.

### Solution

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train = x_train.reshape( x_train.shape[ 0 ], -1 )
x_test = x_test.reshape( x_test.shape[ 0 ], -1 )


### Training:

You can now iterate on your training data in batches. But first we need to transform 

In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(x_train, y_train, epochs=5, batch_size=32)


Evaluate your performance in one line:

In [0]:
model.evaluate(x_test, y_test, batch_size=128)

Or generate predictions on new data:

In [0]:
model.predict(x_test, batch_size=128)

## More than 30 seconds

Let's create a CNN doing classification on MNIST

(taken from: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)

Let's start by importing the functions and classes we want to use.

In [0]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
'''

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Next we are setting some the hyperparameters batch size and epochs.

And some convience information about the data such as image dimensions and the nubmer of classes.

Finally we load the data.

In [0]:
# some hyperparameters 
batch_size = 128
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28
num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print( x_train.shape )

Before we can feed the data to our network we need to perform some transformations.



1.   2 dimensional convolutions expect the image to have multiple color channels our images currently have no channels so we simlpy add a single channel. This is done depending on where keras expects the channel information to be. `channels_first` means the the channel information will be the second axis of our numpy array. `channels_last` means the channel is the last axis. 
2.   Change the dataype to float
3.   Quantize the data to be in $[0,1]$



In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



To perform multiclass classification we need to tranform the labels as well. Currently they are integers $\in [0,9]$

But we need them in one-hot encoded. To transform a number `x` into a one-hot encoded or categorical class label we set the value of an arrray to `1` at index `x`. All other values are `0`. The length of the array is the number of classes.

For example:
`3 -> [ 0, 0, 0, 1, 0, 0, 0, 0, 0, 0 ]`

Thankfully keras does that for us.

In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Next we define our CNN. Note that only the first layer needs and input shape. The shape information for the other layers can be infered from the previous layers.

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')

Keras models need to be compiled ( right now. this probably changed with tensorflow 2). This makes the model ready for execution. We need to pass a loss function, an optimizer and any metrics that we would like to evaluate.

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Now we can train the model. We only need to pass the instance and the labels all other parameters are optional and have reasonalbe default values for most cases. The fit methode will also chop up our data into batches so we can just pass it al.


##!!!! NOTE:
In this case the test data is passed as validation data. If you want to make any claims about your model's performance on unseen test data this is a big no no and is only here for demonstraion.

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Finally we can see how our model does on the test data

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])